In [1]:
import pulp as pl
import pandas as pd
import itertools as it

In [2]:
#imports the data
product_demand = pd.read_excel("/Users/brianblood/Library/CloudStorage/OneDrive-KansasStateUniversity/School/Senior Year/IMSE 633/MSE 633 Warehouse Optimization Data.xlsx",
                             sheet_name='demand',
                             index_col='store')

warehouse_capacity = pd.read_excel("/Users/brianblood/Library/CloudStorage/OneDrive-KansasStateUniversity/School/Senior Year/IMSE 633/MSE 633 Warehouse Optimization Data.xlsx",
                           sheet_name='capacity',
                           index_col='warehouse')

build_cost = pd.read_excel("/Users/brianblood/Library/CloudStorage/OneDrive-KansasStateUniversity/School/Senior Year/IMSE 633/MSE 633 Warehouse Optimization Data.xlsx",
                           sheet_name='build_cost',
                           index_col='warehouse')

ship_cost = pd.read_excel("/Users/brianblood/Library/CloudStorage/OneDrive-KansasStateUniversity/School/Senior Year/IMSE 633/MSE 633 Warehouse Optimization Data.xlsx",
                            sheet_name='cost',
                            header=[0,1],
                            index_col=0)

In [3]:
df_products_stores = ship_cost.columns.tolist()
df_products = ['canned', 'meat']
df_stores = ship_cost['canned'].columns.tolist()
df_warehouses = ship_cost.index.tolist()

In [4]:
class ACME:

    def __init__(self,
                 stores,
                 warehouses,
                 products,
                 demand,
                 cost,
                 build,
                 capacity,
                 extra_constraints = False,
                 solver=pl.CPLEX_PY,
                 name="ACME Warehousing",
                 ):

        #defines all variables and constants used in the problem
        self.solver = solver
        self.stores = stores
        self.warehouses = warehouses
        self.products = products
        self.demand = demand
        self.cost = cost
        self.build_cost = build
        self.capacity = capacity
        self.extra_constraints = extra_constraints

        #defines LP problem
        self.prob = pl.LpProblem(name=name, sense=pl.LpMinimize)

        #creates decision variables

        #product variables
        self.product_vars = pl.LpVariable.dicts("products",
                                             ((w,s,p) for w, s, p in it.product(self.warehouses, self.stores, self.products)),
                                             cat=pl.LpContinuous,
                                             lowBound=0)
        #warehouse binary vars
        self.warehouse_vars = pl.LpVariable.dicts("warehouse_status",
                                             (i for i in self.warehouses),
                                             cat=pl.LpBinary)

    def objective(self):
        #objective function
        self.prob += (pl.lpSum((self.product_vars[w, s, p] * self.cost.loc[w, (p, s)] * 60)
                              for w, s, p in it.product(self.warehouses, self.stores, self.products))
                            + pl.lpSum(self.warehouse_vars[w] * self.build_cost.loc[w, 'build_cost']
                               for w in self.warehouses))
    def constraints(self):

        for p in self.products:
            for s in self.stores:
                #demand
                self.prob += (pl.lpSum(self.product_vars[w, s, p]
                                      for w in self.warehouses) >= self.demand.loc[s, p])
            for w in self.warehouses:
                #whether sites are open or not
                self.prob += (pl.lpSum(self.product_vars[w,s,p] for s in self.stores) <= self.capacity.loc[w, p] * self.warehouse_vars[w])


        if self.extra_constraints:
            self.prob += self.warehouse_vars['W4'] >= self.warehouse_vars['W2']
            self.prob += self.warehouse_vars['W4'] + self.warehouse_vars['W1'] <= 1
            self.prob += self.warehouse_vars['W2'] + self.warehouse_vars['W3'] >= self.warehouse_vars['W1']

    def build(self):
        #build the model
        self.objective()
        self.constraints()

    def solve(self):
        #solve the model
        self.prob.solve(self.solver)
        return self.prob.objective.value()

In [5]:
#building the optimal model
path_to_cplex = "/Applications/CPLEX_Studio201/cplex/bin/x86-64_osx/cplex"
baseline_model = ACME(stores=df_stores,
                      warehouses=df_warehouses,
                      products=df_products,
                      demand=product_demand,
                      cost=ship_cost,
                      build=build_cost,
                      capacity=warehouse_capacity,
                      solver=pl.CPLEX_CMD(path=path_to_cplex, mip=True))

/Users/brianblood/opt/anaconda3/envs/hanyang-ml/lib/python3.8/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [6]:
#solve the model and print objective value
baseline_model.build()
baseline_objective = baseline_model.solve()
print(baseline_objective)


Welcome to IBM(R) ILOG(R) CPLEX(R) Interactive Optimizer 20.1.0.0
  with Simplex, Mixed Integer & Barrier Optimizers
5725-A06 5725-A29 5724-Y48 5724-Y49 5724-Y54 5724-Y55 5655-Y21
Copyright IBM Corp. 1988, 2020.  All Rights Reserved.

Type 'help' for a list of available commands.
Type 'help' followed by a command name for more
information on commands.

CPLEX> Problem '/var/folders/7x/vvlz820d5l9d2yr0jb1jggmc0000gn/T/145aba2371264babb4f0f031f9469601-pulp.lp' read.
Read time = 0.01 sec. (0.00 ticks)
CPLEX> Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
Tried aggregator 1 time.
Reduced MIP has 22 rows, 60 columns, and 120 nonzeros.
Reduced MIP has 4 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (0.05 ticks)
Found incumbent of value 2.2449480e+07 after 0.04 sec. (0.16 ticks)
Probing time = 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 22 rows, 60 columns, and 120 nonzeros.
Reduced MIP has 4 binaries, 0 generals, 

In [7]:
# change build cost to reflect change in Excel sheet
build_cost_2 = build_cost
build_cost_2.loc['W1', 'build_cost'] = 4900000

# change product demand to reflect change in Excel sheet
product_demand_2 = product_demand
product_demand_2.loc['S1', 'canned'] = 28000
product_demand_2.loc['S1', 'meat'] = 18000

In [8]:
warehouse_model = ACME(stores=df_stores,
                      warehouses=df_warehouses,
                      products=df_products,
                      demand=product_demand_2,
                      cost=ship_cost,
                      build=build_cost_2,
                      capacity=warehouse_capacity,
                      extra_constraints=True,
                      solver=pl.CPLEX_CMD(path=path_to_cplex, mip=True))

In [9]:
warehouse_model.build()
warehouse_objective = warehouse_model.solve()
print(warehouse_objective)


Welcome to IBM(R) ILOG(R) CPLEX(R) Interactive Optimizer 20.1.0.0
  with Simplex, Mixed Integer & Barrier Optimizers
5725-A06 5725-A29 5724-Y48 5724-Y49 5724-Y54 5724-Y55 5655-Y21
Copyright IBM Corp. 1988, 2020.  All Rights Reserved.

Type 'help' for a list of available commands.
Type 'help' followed by a command name for more
information on commands.

CPLEX> Problem '/var/folders/7x/vvlz820d5l9d2yr0jb1jggmc0000gn/T/4195de11941e43bc967e6abd6d51957a-pulp.lp' read.
Read time = 0.00 sec. (0.00 ticks)
CPLEX> Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
Tried aggregator 1 time.
Reduced MIP has 25 rows, 60 columns, and 127 nonzeros.
Reduced MIP has 4 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.05 ticks)
Probing changed sense of 1 constraints.
Probing time = 0.00 sec. (0.02 ticks)
Tried aggregator 2 times.
Detecting symmetries...
MIP Presolve modified 1 coefficients.
Aggregator did 1 substitutions.
Reduced MIP has 24 rows, 59 columns, and 124 nonzero

In [10]:
# changes values based on operational changes
build_cost_lower = build_cost * .75
warehouse_capacity_higher = warehouse_capacity * 1.2

In [11]:
#make the price change model
price_change_model = ACME(stores=df_stores,
                        warehouses=df_warehouses,
                        products=df_products,
                        demand=product_demand,
                        cost=ship_cost,
                        build=build_cost_lower,
                        capacity=warehouse_capacity_higher,
                        solver=pl.CPLEX_CMD(path=path_to_cplex, mip=True))

In [12]:
#solve the business plan model and print the objective
price_change_model.build()
price_objective = price_change_model.solve()
print(price_objective)


Welcome to IBM(R) ILOG(R) CPLEX(R) Interactive Optimizer 20.1.0.0
  with Simplex, Mixed Integer & Barrier Optimizers
5725-A06 5725-A29 5724-Y48 5724-Y49 5724-Y54 5724-Y55 5655-Y21
Copyright IBM Corp. 1988, 2020.  All Rights Reserved.

Type 'help' for a list of available commands.
Type 'help' followed by a command name for more
information on commands.

CPLEX> Problem '/var/folders/7x/vvlz820d5l9d2yr0jb1jggmc0000gn/T/371f274a508d4ffd9a2d374682336919-pulp.lp' read.
Read time = 0.00 sec. (0.00 ticks)
CPLEX> Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
Tried aggregator 1 time.
Reduced MIP has 22 rows, 60 columns, and 120 nonzeros.
Reduced MIP has 4 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.05 ticks)
Found incumbent of value 1.7954040e+07 after 0.00 sec. (0.15 ticks)
Probing time = 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 22 rows, 60 columns, and 120 nonzeros.
Reduced MIP has 4 binaries, 0 generals, 

In [13]:
#output the results from the decision variables
baseline_output = []
for w, s, p in it.product(df_warehouses, df_stores, df_products):
    var_output = {
        'Warehouse': w,
        'Store': s,
        'Product': p,
        'Pounds': baseline_model.product_vars[w,s,p].varValue,
        'Warehouse Status': baseline_model.warehouse_vars[w].varValue
    }
    baseline_output.append(var_output)

plan_output = []
for w, s, p in it.product(df_warehouses, df_stores, df_products):
    var_output = {
        'Warehouse': w,
        'Store': s,
        'Product': p,
        'Pounds': price_change_model.product_vars[w,s,p].varValue,
        'Warehouse Status': price_change_model.warehouse_vars[w].varValue
    }
    plan_output.append(var_output)

warehouse_output = []
for w, s, p in it.product(df_warehouses, df_stores, df_products):
    var_output = {
        'Warehouse': w,
        'Store': s,
        'Product': p,
        'Pounds': warehouse_model.product_vars[w,s,p].varValue,
        'Warehouse Status': warehouse_model.warehouse_vars[w].varValue
    }
    warehouse_output.append(var_output)

In [14]:
df_baseline = pd.DataFrame.from_records(baseline_output)
df_plan = pd.DataFrame.from_records(plan_output)
df_warehouse = pd.DataFrame.from_records(warehouse_output)

In [15]:
df_warehouse

,Warehouse,Store,Product,Pounds,Warehouse Status
0,W1,S1,canned,0.0,0.0
1,W1,S1,meat,0.0,0.0
2,W1,S2,canned,0.0,0.0
3,W1,S2,meat,0.0,0.0
4,W1,S3,canned,0.0,0.0
5,W1,S3,meat,0.0,0.0
6,W1,S4,canned,0.0,0.0
7,W1,S4,meat,0.0,0.0
8,W1,S5,canned,0.0,0.0
9,W1,S5,meat,0.0,0.0


In [16]:
df_plan

,Warehouse,Store,Product,Pounds,Warehouse Status
0,W1,S1,canned,28000.0,1.0
1,W1,S1,meat,18000.0,1.0
2,W1,S2,canned,0.0,1.0
3,W1,S2,meat,0.0,1.0
4,W1,S3,canned,23000.0,1.0
5,W1,S3,meat,16300.0,1.0
6,W1,S4,canned,24880.0,1.0
7,W1,S4,meat,14200.0,1.0
8,W1,S5,canned,4600.0,1.0
9,W1,S5,meat,0.0,1.0


In [17]:
df_baseline

,Warehouse,Store,Product,Pounds,Warehouse Status
0,W1,S1,canned,24000.0,1.0
1,W1,S1,meat,12600.0,1.0
2,W1,S2,canned,0.0,1.0
3,W1,S2,meat,0.0,1.0
4,W1,S3,canned,6920.0,1.0
5,W1,S3,meat,16300.0,1.0
6,W1,S4,canned,24880.0,1.0
7,W1,S4,meat,14200.0,1.0
8,W1,S5,canned,0.0,1.0
9,W1,S5,meat,0.0,1.0


In [18]:
# write the results of the model to excel for the answer
with pd.ExcelWriter('/Users/brianblood/Downloads/IMSE 633 Warehouse Output VSRN 2 .xlsx') as writer:
    #df_baseline.to_excel(writer, sheet_name='Baseline')
    #df_plan.to_excel(writer, sheet_name='Plan')
    df_warehouse.to_excel(writer, sheet_name='Warehouse')